Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [10]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

import warnings 
warnings.filterwarnings("ignore") 

import xgboost as xgb

import spacy
nlp = spacy.load("en_core_web_sm")

from nltk.corpus import stopwords as nltk_stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#загрузим датасет
df = pd.read_csv('/home/andrey/Загрузки/toxic_comments.csv')

#посмотрим на датасет
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df[df['toxic'] == 1].count() / df[df['toxic'] == 0].count()

text     0.113188
toxic    0.113188
dtype: float64

Токсичных комментариев в выборке примерно 11%

Данные состоят из твитов на английском языке лемматизируем их с помощью spacy

In [9]:
#лемматизируем текст
corpus = df['text'].values

lemmed_text = []
for i in range(len(corpus)):
    text = nlp(corpus[i])
    string = []
    for token in text:
        string.append(token.lemma_)
    lemmed_text.append(string)

In [11]:
#избавимся от лишних символов с помощью регулярных выражений

re_text = []

for sen in range(0, len(lemmed_text)):
    
    document = re.sub(r'\W', ' ', str(lemmed_text[sen]))
    
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    document = re.sub(r'\s+', ' ', document, flags=re.I)

    document = re.sub(r'^b\s+', '', document)
    
    document = document.lower()
    
    re_text.append(document)

In [3]:
Z = pd.read_csv('/home/andrey/Загрузки/re_text')

In [4]:
re_text = Z['re_text']

# 2. Обучение

In [5]:
#разделим выборку на train, test и valid
y = df['toxic']
X = re_text

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [6]:
#применим downsampling для решения проблемы дисбаланса классов.
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

X_train, y_train = downsample(X_train, y_train, 0.6)

In [7]:
#векторизируем текст
tokenizer = nltk.casual.TweetTokenizer(preserve_case=False, reduce_len=True)
count_vect = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=nltk_stopwords.words('english'),
                             max_features = 5000) 

In [8]:
X_train = count_vect.fit_transform(X_train).toarray()
X_test = count_vect.transform(X_test).toarray()

In [11]:
%%time
#logistic regression
model = LogisticRegression(random_state=12345)

param = {"class_weight" : ['balanced', None],
         "penalty" : ["l1", "l2", "elasticnet", "None"],
         "solver" : ["liblinear", "lbfgs"]
        }

model_rscv = RandomizedSearchCV(model, param_distributions = param, scoring = "f1",
                             cv = 2, verbose = 0, random_state = 12345, n_jobs = 2, n_iter = 10)

model_lr = model_rscv.fit(X_train, y_train)

print("Лучшие гиперпараметры для модели логистической регрессии")
print()
print(model_lr.best_estimator_.get_params())

pred = model_lr.predict(X_test)
print()
print(metrics.classification_report(y_test, pred))
print()
print("F1 = {}".format(f1_score(y_test, pred)))

Лучшие гиперпараметры для модели логистической регрессии

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 12345, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     35806
           1       0.81      0.72      0.76      4087

    accuracy                           0.95     39893
   macro avg       0.89      0.85      0.87     39893
weighted avg       0.95      0.95      0.95     39893


F1 = 0.760274860624919
CPU times: user 59.9 s, sys: 1.86 s, total: 1min 1s
Wall time: 4min 10s


In [28]:
%%time
#обучим модель случайного леса с перебором гиперпараметров

for  i in range(100, 200, 100):
    m = RandomForestClassifier(n_estimators = i, random_state=12345, n_jobs = -1)

    m.fit(X_train, y_train)
    predicts = m.predict(X_test)
    print('estim:', i)
    print(metrics.classification_report(y_test, predicts), metrics.f1_score(y_test, predicts))

estim: 100
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     35806
           1       0.78      0.73      0.75      4087

    accuracy                           0.95     39893
   macro avg       0.88      0.85      0.86     39893
weighted avg       0.95      0.95      0.95     39893
 0.7546261089987326
CPU times: user 17min 4s, sys: 1.52 s, total: 17min 6s
Wall time: 1min 29s


In [25]:
xgb_clf = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)

parameters = {"eta" : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
     "max_depth" : [3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [1, 3, 5, 7],
     "gamma" : [0.0, 0.1, 0.2 , 0.3, 0.4],
     "colsample_bytree" : [0.3, 0.4, 0.5 , 0.7],
     "n_estimators": [1000, 2000, 3000]}


xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "f1",
                             cv = 4, verbose = 3, random_state = 12345, n_jobs = 1)


# Fit the model
model_xgboost = xgb_rscv.fit(X_train, y_train)

best_par = model_xgboost.best_estimator_.get_params()

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4, score=0.795, total=  40.5s
[CV] n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.5s remaining:    0.0s


[CV]  n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4, score=0.795, total=  40.0s
[CV] n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV]  n_estimators=500, min_child_weight=1, max_depth=12, gamma=0.0, eta=0.1, colsample_bytree=0.4, score=0.796, total=  39.1s
[CV] n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7 
[CV]  n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7, score=0.747, total=  35.9s
[CV] n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7 
[CV]  n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7, score=0.754, total=  35.4s
[CV] n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7 
[CV]  n_estimators=300, min_child_weight=3, max_depth=12, gamma=0.1, eta=0.05, colsample_bytree=0.7, score=0.748, total=  34.8s
[CV] n_estimators=300, min_child_weight=1, max_depth=3, gamma=0.2, eta=0.2, colsample_bytree=0.3 
[CV]  n_estimators=300, min_child_weight=1, max_depth=3, gamma=0.2, eta=0.2, colsample_bytr

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 13.2min finished


In [27]:
preds = model_xgboost.predict(X_test)

print(metrics.classification_report(y_test, preds))
print("F1 = {}".format(f1_score(y_test, preds)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     35806
           1       0.84      0.72      0.78      4087

    accuracy                           0.96     39893
   macro avg       0.90      0.85      0.88     39893
weighted avg       0.96      0.96      0.96     39893

F1 = 0.7763590891141239


На модели логистической регрессии удалось достичь значение метрики F1  0.760274860624919

У модели случайного леса с 100 деревьями удалось достичь значение метрики F1 0.7546261089987326

Модель градиентно бустинга достигла значения F1 0.7763590891141239

# 3. Выводы

Модель градиентного бустинга показала лучший результат метрики F1 на валидационной выборке, на тестовой выборке удалось достичь показателся F1: 0.7763590891141239

Так же по сравнению со случайным лесом модель обучается быстрее и показывает лучший скор.